### Load Model

In [1]:
import numpy as np
import pandas as pd
from sklearn.externals import joblib
from keras.models import model_from_json

Using TensorFlow backend.


In [2]:
from keras.models import load_model
loaded_model = load_model('model/MLP_model.h5')

In [2]:
def load_model():
    model = model_from_json(open('model/model_architecture.json').read())
    model.load_weights('model/model_weights.h5')
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

loaded_model = load_model()

### 整理要預測的資料

### Input:
* loan_amt： 預借金額，以1000為倍數，最少1千美金，最多4萬美金
* term： 還款時間，限36或是60個月
* installment： 每月還多少，loan_amt/ term ，可能不會作為feature
* loan_status： 錢借到後有沒有準時還款(從擷取資料的時間點來看，因此不會影響grade)
* home_ownership： 方屋所有權，分為租的、自有、有房貸
* purpose： 借款目的
* verification_status：
* emp_length： 工作資歷
* annual_inc： 年收入

### 1. Load scaler & encoder

#### Load Encoder

In [3]:
def feature_encode(df):
    features = ['term', 'home_ownership', 'loan_status', 'purpose', 'verification_status', 'emp_length']
    for feature in features:
        print(feature)
        le = joblib.load('encoder/le_%s.pkl' % feature)
        print(le.classes_)
        print(df[feature])
        df[feature] = le.transform(df[feature]) # Transform Categories Into Integers
        ohe = joblib.load('encoder/ohe_%s.pkl' % feature)
        featured = ohe.transform(df[[feature]])
        print(featured)
        feature_array = featured.toarray()
        feature_labels = list(le.classes_)
        one_hot_features = pd.DataFrame(feature_array, columns=feature_labels)
        df = pd.concat([df, one_hot_features], axis=1)
        df = df.drop(feature, axis=1)
    return df

In [4]:
input_list = [[2500, 30000, ' 60 months', 'RENT', 'Charged Off', 'car', 'Source Verified', '< 1 year']]
fake_new_input = pd.DataFrame(input_list, columns=['loan_amt', 'annual_inc', 'term', 'home_ownership', 'loan_status', 'purpose', 'verification_status', 'emp_length'])
fake_new_input

,loan_amt,annual_inc,term,home_ownership,loan_status,purpose,verification_status,emp_length
0,2500,30000,60 months,RENT,Charged Off,car,Source Verified,< 1 year


In [5]:
predict_X = feature_encode(fake_new_input)

term
[' 36 months' ' 60 months']
0     60 months
Name: term, dtype: object
  (0, 1)	1.0
home_ownership
['ANY' 'MORTGAGE' 'NONE' 'OTHER' 'OWN' 'RENT']
0    RENT
Name: home_ownership, dtype: object
  (0, 5)	1.0
loan_status
['Charged Off' 'Current' 'Default'
 'Does not meet the credit policy. Status:Charged Off'
 'Does not meet the credit policy. Status:Fully Paid' 'Fully Paid'
 'In Grace Period' 'Issued' 'Late (16-30 days)' 'Late (31-120 days)']
0    Charged Off
Name: loan_status, dtype: object
  (0, 0)	1.0
purpose
['car' 'credit_card' 'debt_consolidation' 'educational' 'home_improvement'
 'house' 'major_purchase' 'medical' 'moving' 'other' 'renewable_energy'
 'small_business' 'vacation' 'wedding']
0    car
Name: purpose, dtype: object
  (0, 0)	1.0
verification_status
['Not Verified' 'Source Verified' 'Verified']
0    Source Verified
Name: verification_status, dtype: object
  (0, 1)	1.0
emp_length
['1 year' '10+ years' '2 years' '3 years' '4 years' '5 years' '6 years'
 '7 years' '8 years

In [6]:
predict_X

,loan_amt,annual_inc,36 months,60 months,ANY,MORTGAGE,NONE,OTHER,OWN,RENT,...,2 years,3 years,4 years,5 years,6 years,7 years,8 years,9 years,< 1 year,nan
0,2500,30000,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [ ]:
# reverse one hot encode to label 
# https://stackoverflow.com/questions/22548731/how-to-reverse-sklearn-onehotencoder-transform-to-recover-original-data

#### Load scaler

In [6]:
predict_X = predict_X.values
minMax = joblib.load("scaler/XminMAX.pkl")
predict_X = minMax.transform(predict_X)
predict_X

array([[0.05797101, 0.00315789, 0.        , 1.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 1.        ,
        1.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        1.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        1.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 1.        , 0.        ]])

In [6]:
predict_X.shape

(1, 49)

### 2. Predict

In [7]:
predictions = loaded_model.predict_classes(predict_X)
predictions

array([10], dtype=int64)

In [8]:
# 分出的類別
predictions[0]

10

In [9]:
# 各類別機率
predict_result = loaded_model.predict(predict_X)
predict_result

array([[0.2696081 , 0.26962972, 0.27019373, 0.27097145, 0.2719433 ,
        0.27223343, 0.2720921 , 0.27293515, 0.27275488, 0.27238423,
        0.273021  , 0.27224192, 0.27164817, 0.2717831 , 0.2713419 ,
        0.2709789 , 0.27012128, 0.27032793, 0.27014434, 0.26947153,
        0.2691223 , 0.2693926 , 0.26918077, 0.26839268, 0.26850575,
        0.26864254, 0.26788968, 0.268238  , 0.2685092 , 0.26815683,
        0.26806173, 0.2682221 , 0.26810804, 0.2680526 , 0.2683508 ]],
      dtype=float32)

In [10]:
max(predict_result[0])

0.273021